In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import explained_variance_score
from parameters import Parameters
from methods import Methods
from functions import *
from errors import *
from random import random

In [ ]:
np.seterr(all='ignore')

In [ ]:
# load dataset
df = pd.read_csv('btc_usd.csv')
df.head()
# df = df.to_numpy()
# X = df[:, [i for i in range(10)]]
# y = df[:, 10]

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['PastWeekOpen'] = df['Close'].shift(1)
df['PastWeekHigh'] = df['High'].shift(0)
df['PastWeekLow'] = df['Low'].shift(0)
# df = df.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
df = df.drop(columns=['Open', 'High', 'Low', 'Adj Close'])
df.head()

In [ ]:
train_mask = (df['Date'] >= '2021-6-1') & (df['Date'] <= '2021-12-31')
train_df = df.loc[train_mask]
train_df.head()

In [ ]:
test_mask = (df['Date'] >= '2022-1-1') & (df['Date'] <= '2022-2-19')
test_df = df.loc[test_mask]
test_df.head()

In [ ]:
X_train = train_df[['PastWeekOpen', 'PastWeekHigh', 'PastWeekLow']]
# X_train = train_df[['PastWeekHigh', 'PastWeekLow', 'Volume']]
y_train = train_df['Close']
date_train = train_df['Date']
X_test = test_df[['PastWeekOpen', 'PastWeekHigh', 'PastWeekLow']]
# X_test = test_df[['PastWeekHigh', 'PastWeekLow', 'Volume']]
y_test = test_df['Close']
date_test = test_df['Date']

In [ ]:
# X_train.head()
# y_train.head()
# X_test.head()
y_test.head()

In [ ]:
# convert to numpy objects
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
Parameters.CONSTANTS = [-5, 5]
# Parameters.CONSTANTS = [1, 10]
Parameters.FEATURES = X_train.shape[1]
Parameters.CONSTANTS_TYPE = 'range'
# Parameters.CONSTANTS_TYPE = 'integer'
# expressions = [Add, Sub, Mul, Div, Sin, Cos, Rlog, Exp, Pow]
expressions = [Add, Sub, Mul, Div]
terminals = [Variable, Constant]

In [ ]:
from fp import FP
model = FP(pop_size=50,
        alpha=0.1,
        beta=0.5,
        gamma=1.0,
        max_evaluations=2000,
        initial_min_depth=0,
        initial_max_depth=6,
        max_depth=15,
        error_function=RMSE,
        expressions=expressions,
        terminals=terminals,
        target_error=0,
        verbose=True
        )

model.fit(X_train, y_train)

In [ ]:
y_fit = model.predict(X_train)
y_pred = model.predict(X_test)
train_score = explained_variance_score(y_train, y_fit)
test_score = explained_variance_score(y_test, y_pred)
print('Train score: {}'.format(train_score))
print('Test score: {}'.format(test_score))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
pylab.rcParams['figure.figsize'] = (20, 8)
ax = plt.axes()
ax.grid(linestyle=':', linewidth=1, alpha=1, zorder=1)
plt.ylabel("Price ($)")
line = [None, None, None, None]
line[0], = ax.plot(date_train, y_train, linestyle=':', color='black', linewidth=0.7, zorder=2, label='Targeted')    
line[1], = ax.plot(date_train, y_fit, linestyle='-', color='red', linewidth=0.7, zorder=3, label='Trained')
line[2], = ax.plot(date_test, y_test, linestyle=':', color='black', linewidth=0.7, zorder=2)
line[3], = ax.plot(date_test, y_pred, linestyle='-', color='blue', linewidth=0.7, zorder=3, label='Predected')
plt.draw()
plt.legend()
plt.show()

In [ ]:
from dfp import DFP
dfp_model = DFP(pop_size=50,
        alpha=0.01,
        beta=0.05,
        gamma=0.1,
        max_evaluations=20000,
        initial_min_depth=0,
        initial_max_depth=6,
        max_depth=15,
        error_function=IR2,
        expressions=expressions,
        terminals=terminals,
        target_error=0,
        verbose=True
        )

dfp_model.fit(X_train, y_train)

In [ ]:
dfp_y_fit = dfp_model.predict(X_train)
dfp_y_pred = dfp_model.predict(X_test)
dfp_train_score = explained_variance_score(y_train, dfp_y_fit)
dfp_test_score = explained_variance_score(y_test, dfp_y_pred)
print('Train score: {}'.format(dfp_train_score))
print('Test score: {}'.format(dfp_test_score))

In [ ]:
# example of calculate the mean squared error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_poisson_deviance

print('Train RMSE: ', mean_squared_error(y_train, dfp_y_fit, squared=False) )
print('Train R2: ', r2_score(y_train, dfp_y_fit) )

print('Test RMSE: ', mean_squared_error(y_test, dfp_y_pred, squared=False) )
print('Test R2: ', r2_score(y_test, dfp_y_pred) )


In [ ]:
ax = plt.axes()
ax.grid(linestyle=':', linewidth=1, alpha=1, zorder=1)
plt.ylabel("Price ($)")
line = [None, None, None, None]
line[0], = ax.plot(date_train, y_train, linestyle=':', color='black', linewidth=0.7, zorder=2, label='Targeted')    
line[1], = ax.plot(date_train, dfp_y_fit, linestyle='-', color='red', linewidth=0.7, zorder=3, label='Trained')
line[2], = ax.plot(date_test, y_test, linestyle=':', color='black', linewidth=0.7, zorder=2)
line[3], = ax.plot(date_test, dfp_y_pred, linestyle='-', color='blue', linewidth=0.7, zorder=3, label='Predected')
plt.axvline(x=date_test.iloc[0], linestyle='-', color='black', linewidth='1')
plt.draw()
plt.legend()
plt.show()